In [1]:
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

# os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

docs = [
    Document(
        page_content="Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of experience in professional athletics, Marcus established the gym to provide personalized fitness solutions for people of all levels. The gym spans 10,000 square feet and features state-of-the-art equipment.",
        metadata={"source": "about.txt"}
    ),
    Document(
        page_content="Peak Performance Gym is open Monday through Friday from 5:00 AM to 11:00 PM. On weekends, our hours are 7:00 AM to 9:00 PM. We remain closed on major national holidays. Members with Premium access can enter using their key cards 24/7, including holidays.",
        metadata={"source": "hours.txt"}
    ),
    Document(
        page_content="Our membership plans include: Basic (₹1,500/month) with access to gym floor and basic equipment; Standard (₹2,500/month) adds group classes and locker facilities; Premium (₹4,000/month) includes 24/7 access, personal training sessions, and spa facilities. We offer student and senior citizen discounts of 15% on all plans. Corporate partnerships are available for companies with 10+ employees joining.",
        metadata={"source": "membership.txt"}
    ),
    Document(
        page_content="Group fitness classes at Peak Performance Gym include Yoga (beginner, intermediate, advanced), HIIT, Zumba, Spin Cycling, CrossFit, and Pilates. Beginner classes are held every Monday and Wednesday at 6:00 PM. Intermediate and advanced classes are scheduled throughout the week. The full schedule is available on our mobile app or at the reception desk.",
        metadata={"source": "classes.txt"}
    ),
    Document(
        page_content="Personal trainers at Peak Performance Gym are all certified professionals with minimum 5 years of experience. Each new member receives a complimentary fitness assessment and one free session with a trainer. Our head trainer, Neha Kapoor, specializes in rehabilitation fitness and sports-specific training. Personal training sessions can be booked individually (₹800/session) or in packages of 10 (₹7,000) or 20 (₹13,000).",
        metadata={"source": "trainers.txt"}
    ),
    Document(
        page_content="Peak Performance Gym's facilities include a cardio zone with 30+ machines, strength training area, functional fitness space, dedicated yoga studio, spin class room, swimming pool (25m), sauna and steam rooms, juice bar, and locker rooms with shower facilities. Our equipment is replaced or upgraded every 3 years to ensure members have access to the latest fitness technology.",
        metadata={"source": "facilities.txt"}
    )
]

db = Chroma.from_documents(docs, embeddings)


/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [2]:
retriever = db.as_retriever(search_type="mmr", search_kwargs = {"k": 3})

In vector-based retrieval, search_type controls how documents are selected. The common types are "similarity", which simply returns the top-k documents closest to the query by similarity score (fast and straightforward); "mmr" (Maximal Marginal Relevance), which balances similarity with diversity to avoid repetitive results, useful for richer answers; and "similarity_score_threshold", which only returns documents if their similarity exceeds a defined threshold, ensuring quality over quantity. These three cover most real-world needs. Some libraries or setups might also allow custom or hybrid strategies, but "similarity", "mmr", and "similarity_score_threshold" are the main officially supported ones across LangChain and major vector stores like FAISS, Chroma, and Pinecone.



In [4]:
retriever.invoke("Who founded the gym and what are the timings?")

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6


[Document(metadata={'source': 'about.txt'}, page_content='Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of experience in professional athletics, Marcus established the gym to provide personalized fitness solutions for people of all levels. The gym spans 10,000 square feet and features state-of-the-art equipment.'),
 Document(metadata={'source': 'classes.txt'}, page_content='Group fitness classes at Peak Performance Gym include Yoga (beginner, intermediate, advanced), HIIT, Zumba, Spin Cycling, CrossFit, and Pilates. Beginner classes are held every Monday and Wednesday at 6:00 PM. Intermediate and advanced classes are scheduled throughout the week. The full schedule is available on our mobile app or at the reception desk.'),
 Document(metadata={'source': 'membership.txt'}, page_content='Our membership plans include: Basic (₹1,500/month) with access to gym floor and basic equipment; Standard (₹2,500/month) adds group classes and locker

In [5]:
from langchain_core.prompts import ChatPromptTemplate

template = """ 
Answer the question based only on the following context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

# llm=ChatGroq(model_name="gemma2-9b-it")
# llm = ChatGroq(model="llama-3.1-8b-instant")
llm=ChatGroq(model_name="llama-3.3-70b-versatile")


def format_docs(docs): 
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x)), 
        "question": lambda x: x
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

qa_chain.invoke("Who is the founder and what are the timings?")

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6


'The founder of Peak Performance Gym is Marcus Chen, a former Olympic athlete. \n\nThe timings are not explicitly mentioned for all members, but Premium members have 24/7 access to the gym. The timings for other members (Basic and Standard) are not specified.'